# Rendimiento de consultas sobre 10 Terabytes en tablas Iceberg gestionadas por Snowflake

## Tutorial TPC-DS: Micro-particiones y Data Clustering

---

### Bienvenido

Este notebook es un tutorial práctico que demuestra las capacidades de rendimiento de Snowflake al ejecutar consultas analíticas complejas sobre un dataset de **10 Terabytes en formato Iceberg** utilizando el benchmark estándar de la industria **TPC-DS**.

**Objetivos de aprendizaje:**
1. Comprender el modelo de datos TPC-DS en formato Iceberg
2. Entender las **micro-particiones** de Snowflake y su impacto en el rendimiento
3. Dominar el **clustering de datos** para optimizar consultas en tablas Iceberg
4. Ejecutar y analizar consultas del benchmark TPC-DS
5. Medir y comparar el rendimiento con diferentes estrategias de optimización
6. Comparar el comportamiento de tablas Iceberg vs formato nativo Snowflake

**Referencias:**
- [TPC-DS Sample Data](https://docs.snowflake.com/en/user-guide/sample-data-tpcds)
- [TPC-DS Queries (99 queries)](https://docs.snowflake.com/en/_downloads/0eec2c68e78863a07eb994c85e76b188/tpc-ds-all-queries.sql)

---

## 📋 Índice

1. [Introducción al TPC-DS con Tablas Iceberg](#1)
2. [Configuración del Entorno](#2)
3. [Micro-particiones en Snowflake](#3)
4. [Data Clustering: Conceptos y Estrategias](#4)
5. [Dataset TPC-DS 10TB Iceberg](#5)
6. [Consultas de Benchmark](#6)
7. [Mejores Prácticas](#10)


## 1. Introducción al TPC-DS con Tablas Iceberg

**TPC-DS** (Transaction Processing Performance Council - Decision Support) es un benchmark estándar de la industria que modela un sistema de data warehouse de múltiples dimensiones.

### Características del TPC-DS:
- **24 tablas** (7 fact tables + 17 dimensions)
- **99 consultas analíticas complejas**
- Simula escenarios realistas de análisis de datos
- Incluye JOINs complejos, agregaciones, subconsultas y CTEs

### TPC-DS 10TB en formato Iceberg:

Snowflake ofrece el dataset TPC-DS en dos formatos:
1. **Formato nativo Snowflake** (`TPCDS_SF10TCL`)
2. **Managed Iceberg Tables** (`TPCDS_SF10TCL` en Iceberg) ← **Usaremos este**

### ¿Por qué Iceberg?

- **Formato abierto**: Apache Iceberg es un estándar open-source
- **Interoperabilidad**: Los datos pueden ser accedidos por múltiples engines (Spark, Trino, etc.)
- **Time travel**: Historial completo de versiones de datos
- **Schema evolution**: Cambios de schema sin reescribir datos
- **Performance**: Optimizaciones similares a formato nativo Snowflake

### Dataset de 10TB Iceberg:
- Scale Factor: **10,000 GB (10 TB)**
- **65 millones de clientes**
- **400,000+ productos**
- Tabla principal (STORE_SALES): **~29 mil millones de filas**
- Total fact tables: **56+ mil millones de filas**
- **3 canales de venta**: tiendas físicas, catálogo, web
- **5 años de datos** de ventas

![Logo de Snowflake](https://media.licdn.com/dms/image/v2/D5612AQFX40GrSqn35w/article-inline_image-shrink_1000_1488/article-inline_image-shrink_1000_1488/0/1654724687488?e=1760572800&v=beta&t=3rVQ99DdWK2G6xxw_Mr4m59PQ3Ux6aDD1mPBGAzDEZ8)

📚 **Documentación oficial**: [Sample Data: TPC-DS](https://docs.snowflake.com/en/user-guide/sample-data-tpcds)


## 2. Configuración del Entorno

Configuraremos el entorno de Snowflake para trabajar con el dataset TPC-DS 10TB en formato **Managed Iceberg**.

In [ ]:
-- Configuración inicial
-- Desactivar el uso de resultados en caché para mediciones precisas
ALTER SESSION SET USE_CACHED_RESULT = FALSE;

-- Configurar el warehouse para rendimiento óptimo
-- Para 10TB Iceberg se recomienda un warehouse M o superior
USE WAREHOUSE COMPUTE_WH;

ALTER WAREHOUSE SET WAREHOUSE_SIZE = MEDIUM;

-- Verificar el warehouse actual y su tamaño
SELECT 
    CURRENT_WAREHOUSE() AS warehouse_actual,
    CURRENT_VERSION() AS snowflake_version,
    'Recomendado: M o superior para 10TB' AS recomendacion;

### 2.1 Acceso al Dataset TPC-DS Iceberg desde Snowflake Marketplace

Snowflake proporciona el dataset TPC-DS en formato Iceberg directamente desde el Marketplace.

#### Pasos para obtener el dataset:

1. En Snowflake UI, ve a **Data** → **Marketplace**
2. Busca "**TPC-DS 10 TB Managed Iceberg**"
3. Haz clic en **Get**
4. Dale un nombre a la base de datos (`TPCDS_10TB_MANAGED_ICEBERG`)
5. Selecciona el rol que tendrá acceso
6. Haz clic en **Get it for Free**

#### ⚠️ Importante:

- El dataset Iceberg es **físicamente diferente** al formato nativo Snowflake
- Los datos están almacenados en formato **Apache Iceberg**
- Las micro-particiones funcionan de forma similar pero con metadata Iceberg
- Puedes obtener AMBOS datasets (nativo + Iceberg) para comparar rendimiento

**Documentación completa**: [Getting TPC-DS data from Snowflake Marketplace](https://docs.snowflake.com/en/user-guide/sample-data-tpcds#getting-tpc-ds-data-from-snowflake-marketplace)

In [ ]:
-- Configurar la base de datos TPC-DS Iceberg
-- IMPORTANTE: Ajusta el nombre según lo configuraste en el Marketplace
-- Por defecto, Snowflake la crea como TPCDS_10TB_MANAGED_ICEBERG

USE DATABASE TPCDS_10TB_MANAGED_ICEBERG;  -- Ajusta este nombre si es diferente
USE SCHEMA TPCDS_SF10T_ICEBERG;

-- Verificar contexto actual
SELECT 
    CURRENT_DATABASE() AS database_actual,
    CURRENT_SCHEMA() AS schema_actual,
    CURRENT_ROLE() AS rol_actual;

In [ ]:
-- Listar todas las tablas Iceberg disponibles
SHOW TABLES;

-- Verificar que son tablas Iceberg
-- La columna 'kind' debería mostrar 'ICEBERG'

In [ ]:
-- Verificar que estamos usando tablas en formato Iceberg
-- Esto nos da información sobre el tipo de tabla y metadata

SELECT 
    TABLE_CATALOG,
    TABLE_SCHEMA,
    TABLE_NAME,
    TABLE_TYPE,
    IS_ICEBERG,
    ROW_COUNT,
    BYTES
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'TPCDS_SF10T_ICEBERG'
ORDER BY ROW_COUNT DESC NULLS LAST
LIMIT 10;

In [ ]:
-- Verificar el tamaño del dataset Iceberg
-- Conteo de filas en las principales tablas de hechos

SELECT 
    'STORE_SALES' AS tabla,
    COUNT(*) AS num_filas,
    ROUND(COUNT(*) / 1000000000, 2) AS filas_mil_millones
FROM STORE_SALES

UNION ALL

SELECT 
    'CATALOG_SALES' AS tabla,
    COUNT(*) AS num_filas,
    ROUND(COUNT(*) / 1000000000, 2) AS filas_mil_millones
FROM CATALOG_SALES

UNION ALL

SELECT 
    'WEB_SALES' AS tabla,
    COUNT(*) AS num_filas,
    ROUND(COUNT(*) / 1000000000, 2) AS filas_mil_millones
FROM WEB_SALES

ORDER BY num_filas DESC;


## 3. Micro-particiones en Snowflake con Tablas Iceberg

### ¿Qué son las Micro-particiones?

Las **micro-particiones** son la unidad fundamental de almacenamiento de datos en Snowflake. Aunque las tablas Iceberg usan metadata Iceberg, Snowflake las gestiona internamente usando su arquitectura de micro-particiones.

### Características clave:

| Característica | Detalle | En Iceberg |
|----------------|---------|------------|
| **Tamaño** | 50-500 MB sin comprimir | Similar |
| **Formato** | Columnar comprimido | Parquet (Iceberg) |
| **Creación** | Automática | Automática |
| **Inmutabilidad** | No se modifican | Compatible con Iceberg snapshots |
| **Metadatos** | Min/max, null count | + Manifest files Iceberg |

### Ventajas con tablas Iceberg:

1. **Pruning eficiente**: Snowflake + metadata Iceberg = doble capa de optimización
2. **Time travel nativo**: Iceberg snapshots integrados
3. **Partition evolution**: Cambiar estrategia de particionamiento sin reescribir datos
4. **Interoperabilidad**: Mismos datos accesibles desde otros engines

### Arquitectura Iceberg en Snowflake:

```
Tabla Iceberg 10TB
        ↓
Metadata Iceberg (manifest files, snapshots)
        ↓
Micro-particiones Snowflake (Parquet)
        ↓
Query optimizer usa ambos niveles de metadata
        ↓
Pruning ultra-eficiente
```

📚 **Documentación oficial**: 
- [Snowflake Micro-partitions](https://docs.snowflake.com/user-guide/tables-clustering-micropartitions)
- [Iceberg Tables](https://docs.snowflake.com/user-guide/tables-iceberg)

### 3.1 Ejemplo práctico: Visualización de Micro-particiones en Iceberg

Veamos cómo Snowflake usa micro-particiones con tablas Iceberg en una consulta real.

In [ ]:
-- Consulta simple con filtro en fecha sobre tabla Iceberg
-- Snowflake automáticamente hace pruning usando metadata Iceberg + micro-particiones

SELECT 
    SS_SOLD_DATE_SK,
    COUNT(*) AS num_ventas,
    SUM(SS_SALES_PRICE) AS ventas_totales,
    AVG(SS_SALES_PRICE) AS precio_promedio,
    SUM(SS_QUANTITY) AS unidades_vendidas
FROM STORE_SALES
WHERE SS_SOLD_DATE_SK BETWEEN 2451545 AND 2451575  -- Aproximadamente 1 mes
GROUP BY SS_SOLD_DATE_SK
ORDER BY SS_SOLD_DATE_SK
LIMIT 100;

### Analizar el plan de ejecución de la consulta anterior:
- Haz clic en "Query Details" en la UI de Snowflake después de ejecutar

### Observa en el Query Profile:
-   1. "Partitions scanned" (934) vs "Partitions total (82787)"
-   2. Porcentaje de pruning (debería ser muy alto > 98%)
-   3. Bytes escaneados vs bytes totales
-   4. Iceberg metadata used (específico de tablas Iceberg)

![Logo de Snowflake](https://github.com/Snowflake-Spain-SE-Demos-Sandbox/TPC-DS-10-TB-Iceberg-Performance/blob/main/query1.jpg?raw=true)





## 4. Data Clustering: Conceptos y Estrategias en Tablas Iceberg

### ¿Qué es el Clustering en Tablas Iceberg?

El **clustering** en tablas Iceberg funciona de manera similar a las tablas nativas de Snowflake, pero con algunas diferencias importantes:

### Clustering Natural vs Clustering Explícito

| Aspecto | Tablas Nativas | Tablas Iceberg |
|---------|----------------|----------------|
| **Natural** | Orden de inserción | Orden de inserción + Iceberg partitioning |
| **Explícito** | Clustering keys Snowflake | Clustering keys + Iceberg partition spec |
| **Metadata** | Snowflake metadata | Iceberg manifest + Snowflake metadata |
| **Re-clustering** | Automático | Automático (respeta estructura Iceberg) |

### Métricas de Clustering

Snowflake proporciona las mismas funciones de análisis:

- **Clustering Depth**: Profundidad de solapamiento entre micro-particiones
  - Valor bajo (1-4): Excelente clustering
  - Valor alto (>100): Clustering pobre

- **Clustering Ratio**: Porcentaje de datos bien organizados (0-100%)

### Cuándo usar Clustering Keys en Iceberg:

✅ **SÍ usar cuando:**
- Tabla > 1TB (como STORE_SALES con 29B filas)
- Consultas frecuentes con filtros en columnas específicas
- Análisis muestra bajo pruning
- Necesitas optimizar para patrones de acceso específicos

❌ **NO usar cuando:**
- Tabla < 1TB
- Patrones de consulta muy variables
- Ya tienes buen pruning natural (Iceberg puede ser suficiente)

### Ventaja de Iceberg:

Las tablas Iceberg ya vienen con una capa de organización (Iceberg partitioning), por lo que el pruning inicial suele ser mejor que en tablas nativas sin clustering.

📚 **Documentación**: [Clustering Keys](https://docs.snowflake.com/user-guide/tables-clustering-micropartitions)

In [ ]:
-- Analizar información de clustering de STORE_SALES (Iceberg)
-- Esta función funciona igual en tablas Iceberg

SELECT SYSTEM$CLUSTERING_INFORMATION('STORE_SALES') AS clustering_info;

In [ ]:
-- Analizar clustering específico por columna fecha
-- En TPC-DS, la fecha es típicamente la mejor columna para clustering

SELECT SYSTEM$CLUSTERING_INFORMATION('STORE_SALES', '(SS_SOLD_DATE_SK)') AS clustering_por_fecha;

In [ ]:
-- Analizar clustering con múltiples columnas
-- Útil cuando las consultas filtran por fecha Y tienda

SELECT SYSTEM$CLUSTERING_INFORMATION('STORE_SALES', '(SS_SOLD_DATE_SK, SS_STORE_SK)') AS clustering_fecha_tienda;


## 5. Exploración del Dataset TPC-DS 10TB Iceberg

Exploremos la estructura y características del dataset.

In [ ]:
-- Tablas de hechos (Fact Tables) en TPC-DS
SELECT 
    'STORE_SALES' AS tabla_tipo_hechos,
    'Ventas en tiendas físicas' AS descripcion,
    '~29 mil millones de filas' AS escala
UNION ALL
SELECT 'CATALOG_SALES', 'Ventas por catálogo', '~14 mil millones'
UNION ALL
SELECT 'WEB_SALES', 'Ventas en línea', '~7 mil millones'
UNION ALL
SELECT 'STORE_RETURNS', 'Devoluciones de tiendas', 'Varía'
UNION ALL
SELECT 'CATALOG_RETURNS', 'Devoluciones de catálogo', 'Varía'
UNION ALL
SELECT 'WEB_RETURNS', 'Devoluciones web', 'Varía'
UNION ALL
SELECT 'INVENTORY', 'Inventario de productos', 'Varía';

In [ ]:
-- Tablas de dimensiones (Dimension Tables) - Principales
SELECT 
    'CUSTOMER' AS tabla_dimension,
    'Información de clientes (65M)' AS descripcion
UNION ALL
SELECT 'DATE_DIM', 'Dimensión temporal (18 años)'
UNION ALL
SELECT 'ITEM', 'Productos/artículos (400K+)'
UNION ALL
SELECT 'STORE', 'Tiendas físicas'
UNION ALL
SELECT 'PROMOTION', 'Promociones de marketing'
UNION ALL
SELECT 'HOUSEHOLD_DEMOGRAPHICS', 'Demografía de hogares'
UNION ALL
SELECT 'CUSTOMER_DEMOGRAPHICS', 'Demografía de clientes'
UNION ALL
SELECT 'WAREHOUSE', 'Almacenes';

In [ ]:
-- Explorar la estructura de la tabla principal STORE_SALES (Iceberg)
DESCRIBE TABLE STORE_SALES;


## 6. Consultas de Benchmark TPC-DS

Ejecutaremos una selección de las 99 consultas del benchmark TPC-DS sobre las tablas **Iceberg**.

### Consultas seleccionadas:

- **Query 96**: Filtros temporales y demográficos (pruning test)
- **Query 7**: Agregaciones con múltiples JOINs
- **Query 61**: Análisis de efectividad de promociones
- **Query 57**: Outliers en call center (CTEs complejos)

### ⏱️ Tiempo estimado por query:
- Warehouse M: 1 minuto
- Warehouse L: 30 segundos  
- Warehouse XL: 15 segundos

📄 [Todas las 99 queries](https://docs.snowflake.com/en/_downloads/0eec2c68e78863a07eb994c85e76b188/tpc-ds-all-queries.sql)

### 6.1 Query 96: Conteo con filtros temporales y demográficos

Esta consulta demuestra:
- Filtrado por múltiples dimensiones (tiempo, demografía, tienda)
- JOINs entre tabla de hechos Iceberg y dimensiones
- Pruning de micro-particiones usando metadata Iceberg

In [ ]:
-- Query 96: Análisis de ventas por hora y demografía sobre tabla Iceberg
SET query_start = CURRENT_TIMESTAMP();

SELECT /* Query 96 - TPC-DS on Iceberg */
    COUNT(*) AS num_transacciones
FROM STORE_SALES
    ,HOUSEHOLD_DEMOGRAPHICS
    ,TIME_DIM
    ,STORE
WHERE SS_SOLD_TIME_SK = TIME_DIM.T_TIME_SK
    AND SS_HDEMO_SK = HOUSEHOLD_DEMOGRAPHICS.HD_DEMO_SK
    AND SS_STORE_SK = S_STORE_SK
    AND TIME_DIM.T_HOUR = 8
    AND TIME_DIM.T_MINUTE >= 30
    AND HOUSEHOLD_DEMOGRAPHICS.HD_DEP_COUNT = 5
    AND STORE.S_STORE_NAME = 'ese'
ORDER BY COUNT(*)
LIMIT 100;

### 6.2 Query 7: Agregaciones con promociones

Esta consulta ejercita:
- Múltiples JOINs (5 tablas)
- Agregaciones (AVG) sobre datos Iceberg
- Filtros complejos en múltiples dimensiones
- GROUP BY con ORDER BY

In [ ]:
-- Query 7: Análisis de items con filtros demográficos y promocionales
SET query_start = CURRENT_TIMESTAMP();

SELECT /* Query 7 - TPC-DS on Iceberg */
    I_ITEM_ID,
    AVG(SS_QUANTITY) AS agg1,
    AVG(SS_LIST_PRICE) AS agg2,
    AVG(SS_COUPON_AMT) AS agg3,
    AVG(SS_SALES_PRICE) AS agg4
FROM STORE_SALES, CUSTOMER_DEMOGRAPHICS, DATE_DIM, ITEM, PROMOTION
WHERE SS_SOLD_DATE_SK = D_DATE_SK
    AND SS_ITEM_SK = I_ITEM_SK
    AND SS_CDEMO_SK = CD_DEMO_SK
    AND SS_PROMO_SK = P_PROMO_SK
    AND CD_GENDER = 'M'
    AND CD_MARITAL_STATUS = 'M'
    AND CD_EDUCATION_STATUS = '4 yr Degree'
    AND (P_CHANNEL_EMAIL = 'N' OR P_CHANNEL_EVENT = 'N')
    AND D_YEAR = 2001
GROUP BY I_ITEM_ID
ORDER BY I_ITEM_ID
LIMIT 100;

### 6.3 Query 61: Análisis de promociones

Esta consulta demuestra:
- Cálculos de ratios y porcentajes
- Subconsultas en el FROM
- Filtros complejos con ORs
- Análisis de efectividad de canales de marketing

In [ ]:
-- Query 61: Efectividad de promociones por canal
SET query_start = CURRENT_TIMESTAMP();

SELECT /* Query 61 - TPC-DS on Iceberg */
    promotions,
    total,
    CAST(promotions AS DECIMAL(17,4)) / NULLIF(CAST(total AS DECIMAL(17,4)), 0) * 100 AS porcentaje_promociones
FROM (
    SELECT SUM(SS_EXT_SALES_PRICE) AS promotions
    FROM STORE_SALES, STORE, PROMOTION, DATE_DIM, CUSTOMER, CUSTOMER_ADDRESS, ITEM
    WHERE SS_SOLD_DATE_SK = D_DATE_SK
        AND SS_STORE_SK = S_STORE_SK
        AND SS_PROMO_SK = P_PROMO_SK
        AND SS_CUSTOMER_SK = C_CUSTOMER_SK
        AND CA_ADDRESS_SK = C_CURRENT_ADDR_SK
        AND SS_ITEM_SK = I_ITEM_SK
        AND CA_GMT_OFFSET = -6
        AND I_CATEGORY = 'Sports'
        AND (P_CHANNEL_DMAIL = 'Y' OR P_CHANNEL_EMAIL = 'Y' OR P_CHANNEL_TV = 'Y')
        AND S_GMT_OFFSET = -6
        AND D_YEAR = 1999
        AND D_MOY = 11
) promotional_sales,
(
    SELECT SUM(SS_EXT_SALES_PRICE) AS total
    FROM STORE_SALES, STORE, DATE_DIM, CUSTOMER, CUSTOMER_ADDRESS, ITEM
    WHERE SS_SOLD_DATE_SK = D_DATE_SK
        AND SS_STORE_SK = S_STORE_SK
        AND SS_CUSTOMER_SK = C_CUSTOMER_SK
        AND CA_ADDRESS_SK = C_CURRENT_ADDR_SK
        AND SS_ITEM_SK = I_ITEM_SK
        AND CA_GMT_OFFSET = -6
        AND I_CATEGORY = 'Sports'
        AND S_GMT_OFFSET = -6
        AND D_YEAR = 1999
        AND D_MOY = 11
) all_sales
ORDER BY promotions, total
LIMIT 100;


## 7. Mejores Prácticas y Conclusiones

### 🎯 Resumen de aprendizajes clave

#### 1. Micro-particiones con Iceberg
- **Doble capa de metadata**: Iceberg manifest files + Snowflake micro-partitions
- **Pruning mejorado**: El pruning suele ser mejor out-of-the-box que tablas nativas
- **Formato Parquet**: Compresión columnar eficiente
- **Time travel nativo**: Snapshots Iceberg integrados

#### 2. Data Clustering en Iceberg
- **Cuándo aplicar**: Tablas > 1TB con patrones predecibles (como STORE_SALES)
- **Columnas ideales**: Fechas (SS_SOLD_DATE_SK), foreign keys frecuentemente usados
- **Mejoras típicas**: 20-70% en tiempo de ejecución
- **Compatibilidad**: Clustering keys de Snowflake + Iceberg partition spec trabajan juntos

#### 3. Escala de 10TB Iceberg
- **Warehouses**: XL o superior recomendado para queries interactivas
- **Performance**: Comparable a tablas nativas Snowflake
- **Interoperabilidad**: Mismos datos accesibles desde Spark, Trino, etc.
- **Evolución**: Schema evolution y partition evolution sin reescribir datos

### 🔍 Iceberg vs Nativo Snowflake: Guía de decisión

```
¿Necesitas interoperabilidad con otros engines?
    ↓ SÍ → Usar Iceberg
    ↓ NO
¿Necesitas time travel extenso y schema evolution?
    ↓ SÍ → Considerar Iceberg
    ↓ NO
¿Performance es crítico y solo usas Snowflake?
    ↓ SÍ → Nativo puede ser ligeramente más rápido
    ↓ NO
¿Quieres formato abierto y estándar?
    ↓ SÍ → Usar Iceberg
    ↓ NO → Ambos funcionan bien
```

### 💡 Conclusión sobre Iceberg:

En TPC-DS 10TB, las tablas Iceberg ofrecen:
- ✅ **Performance comparable** a tablas nativas (~95-105% del tiempo)
- ✅ **Mejor pruning inicial** (metadata Iceberg + Snowflake)
- ✅ **Flexibilidad** para acceso multi-engine
- ✅ **Evolución de schema** sin interrupciones

### 📚 Recursos adicionales

#### Documentación oficial
- [Sample Data: TPC-DS](https://docs.snowflake.com/en/user-guide/sample-data-tpcds)
- [Iceberg Tables in Snowflake](https://docs.snowflake.com/user-guide/tables-iceberg)
- [Micro-partitions & Clustering](https://docs.snowflake.com/user-guide/tables-clustering-micropartitions)
- [TPC-DS Sample Queries (99 queries)](https://docs.snowflake.com/en/_downloads/0eec2c68e78863a07eb994c85e76b188/tpc-ds-all-queries.sql)

#### TPC-DS Benchmark
- [TPC-DS Specification v3.2.0](http://tpc.org/tpc_documents_current_versions/pdf/tpc-ds_v3.2.0.pdf)
- [TPC Benchmark Kit](http://www.tpc.org/tpc_documents_current_versions/current_specifications.asp)

#### Apache Iceberg
- [Apache Iceberg Documentation](https://iceberg.apache.org/docs/latest/)
- [Iceberg Table Format Spec](https://iceberg.apache.org/spec/)

---

## ✅ Conclusión

Has completado el tutorial de **Rendimiento de consultas sobre 10 Terabytes en tablas Iceberg** utilizando el benchmark TPC-DS.

### Lo que has aprendido:
✅ Arquitectura de micro-particiones de Snowflake con Iceberg  
✅ Diferencias entre formato nativo Snowflake y Managed Iceberg  
✅ Estrategias de data clustering para tablas Iceberg  
✅ Ejecución y análisis de consultas TPC-DS en datasets masivos  
✅ Interpretación de métricas de rendimiento y pruning  
✅ Mejores prácticas para optimización de queries en Iceberg  

### Resultados esperados:
- ⚡ **Performance comparable** a tablas nativas Snowflake
- 📉 **Excelente pruning** gracias a doble capa de metadata
- 💰 **Flexibilidad** sin sacrificar rendimiento
- 🎯 **Queries escalables** que mantienen performance en datasets crecientes
- 🔄 **Interoperabilidad** con otros engines de consulta
